In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot

## Load DATA

In [ ]:
PATH = 'data.csv'

df = pd.read_csv(PATH, delimiter=';', index_col='index')
id_vals = df['id'].unique()
df

index
8226                 Current
8227             Temperature
8228                 Voltage
8229       Accelerometer1RMS
8230       Accelerometer2RMS
                ...         
59461      Accelerometer1RMS
59462      Accelerometer2RMS
59463    Volume Flow RateRMS
59464               Pressure
59465           Thermocouple
Name: id, Length: 51240, dtype: object

['Current' 'Temperature' 'Voltage' 'Accelerometer1RMS' 'Accelerometer2RMS'
 'Volume Flow RateRMS' 'Pressure' 'Thermocouple']
